In my assignment, I used following two models:
    1. Word to sentence is implemented by simply averaging word vectors which are given
    2. Word to sentence is implemented by weighted average where weight is derived from tf_idf values of words.
I chose K as sqrt(N) where N is the number of sentences in a document.

Whole code runs in 3 minutes in my local.

In [1]:
import pickle
import os
import numpy as np
import re
from rouge import Rouge

VECTOR_PATH = '../pre_trained_word2vec_model'
ARTICLES_PATH = '../articles/'
GOLD_SUMMARIES_PATH = '../gold_summaries/'

ALL_FILES = os.listdir(os.path.abspath(os.path.join( os.getcwd() , ARTICLES_PATH)))
FILE_NAMES = []
dic = {}
docs_sentences = []

The code in the below cell is taken and edited from https://stackoverflow.com/questions/4576077/python-split-text-on-sentences. It is a rule based sentence finder using regex.

In [2]:
alphabets= "([A-Za-z])"
prefixes = "(Mr|St|Mrs|Ms|Dr|Prof|Capt|Cpt|Lt|Mt|US)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov|me|edu)"
digits = "([0-9])"
punctuations = {'#', '[', '~', '-', ']', '.', '@', '/', "'", '{', '|', ')',
                '(', '*', ',', '`', ';', '$', '%', '\\', '^', '_', '!', '<', ':', '&', '>', '"', '}', '=', '?', '+'}

def split_into_sentences(text):
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    if "e.g." in text: text = text.replace("e.g.","e<prd>g<prd>")
    if "i.e." in text: text = text.replace("i.e.","i<prd>e<prd>")    
    text = re.sub(digits + "[.]" + digits,"\\1<prd>\\2",text)
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    if "..." in text: text = text.replace("...","<prd><prd><prd>")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    return sentences

Takes less than a minute. I simply read articles, then using above rule based splitter i found sentences. I returned a list of sentences for each document.

In [3]:
def parse_documents_to_sentences():
    global FILE_NAMES
    docs_by_sentences = []
    for article_name in FILE_NAMES:
        article = open( os.path.abspath( os.path.join( os.getcwd() , ARTICLES_PATH , article_name ) ) , "r" , encoding='latin1' )
        sentences = [article.readline().strip()] 
        article = article.read()
        sentences.extend( split_into_sentences( article ) )
        docs_by_sentences.append( sentences )
    return docs_by_sentences

Takes a string, removes punctuations by replacing them with space, makes every word lowercase and returns the result.

In [4]:
def tokenize( input ):
    input = input.strip()
    without_punc = ""
    for char in input:
        if char in punctuations:
            without_punc += ' '
        else:
            without_punc += char
    return list(map(lambda x: x.lower(), without_punc.split()))

This is the implementation for the first model, which takes average of the word vectors to represent a sentence. I also keep a global list named *docs_sentences* to keep the original sentences for the summaries. For safety, I consider the words which has no correspondence with the given pre-trained vector model as if they don't exist in the sentence. 

Takes less than a minute in my local.

In [5]:
def sent2vec(docs_by_sentences):
    global FILE_NAMES
    global docs_sentences
    global dic
    docs_as_vectors = []
    for i in range(len(FILE_NAMES)):
        sentences = []
        actual_sentences = []
        for sentence in docs_by_sentences[i]:
            tokens = tokenize( sentence )
            tokens = list(filter(lambda token: token in dic, tokens) )
            if len(tokens) == 0:
                continue
            average = np.zeros(200)
            for token in tokens:
                average += np.array(dic[token])/len(tokens)
            sentences.append(average)
            actual_sentences.append(sentence)
        docs_as_vectors.append( sentences )
        docs_sentences[i] = actual_sentences
    return docs_as_vectors

This is the implementation for the second model, which takes the weighted average of the word vectors to represent a sentence where weights are related to their *tf_idf* values. First I calculate the frequency and inversed document frequencies of words. And then using these I construct a weight formula which is *(1+log(freq))X(log(N/df)+epsilon)*.
Rest of the logic is the same as the first model.

Takes less than a minute in my local.

In [6]:
def sent2vec2(docs_by_sentences):
    global FILE_NAMES
    global docs_sentences
    global dic
    vocabulary = {}
    freq = [{} for i in range(len(FILE_NAMES))]
    for doc_i in range(len(FILE_NAMES)):
        doc = docs_by_sentences[doc_i]
        all_tokens_of_a_doc = []
        for sentence in doc:
            tokens = tokenize( sentence )
            tokens = list(filter(lambda token: token in dic, tokens) )
            all_tokens_of_a_doc.extend(tokens)
        for token in all_tokens_of_a_doc:
            if token not in freq[doc_i]:
                freq[doc_i][token] = 0
            freq[doc_i][token] += 1/len(all_tokens_of_a_doc)
        for token in set(all_tokens_of_a_doc):
            if token not in vocabulary:
                vocabulary[token] = 0
            vocabulary[token] += 1

    docs_as_vectors = []
    for i in range(len(FILE_NAMES)):
        sentences = []
        actual_sentences = []
        for sentence in docs_by_sentences[i]:
            tokens = tokenize( sentence )
            tokens = list(filter(lambda token: token in dic, tokens) )
            if len(tokens) == 0:
                continue
            average = np.zeros(200)
            weights = []
            for token in tokens:
                weights.append( (1+np.log(freq[i][token]))*(np.log(len(FILE_NAMES)/vocabulary[token]+0.000001) ))
            weights = np.ndarray.tolist( np.array(weights)/np.sum(weights) )
            for j in range(len(tokens)):
                token = tokens[j]
                average += np.array(dic[token])*weights[j]
            sentences.append(average)
            actual_sentences.append(sentence)
        docs_as_vectors.append( sentences )
        docs_sentences[i] = actual_sentences
    return docs_as_vectors

In this part I implemented the k-means clustering algorithm. I think it is not necessary to explain the algorithm because i sitrictly followed the algorithm which is tought us in the lecture. I chose K as square root of the number of sentences in the given document. I iterated the algorithm until the sum of the euclidian distances between the old and the new centroids are smaller than 10^-4.

Then for each cluster, I found the sentence which is closest to the centroid of that cluster. I concatanated the strings and returned them.

Takes around 2 minutes in my local.

In [7]:
def cluster(docs_as_vectors):
    global docs_sentences
    extracted_summaries = []
    for mask in range(len(docs_as_vectors)):
        document = docs_as_vectors[mask]
        num_of_sentences = len(document)
        K = int(np.ceil(np.sqrt(num_of_sentences)))
        centroids = np.arange(num_of_sentences)
        np.random.shuffle( centroids)
        centroids = [document[x] for x in centroids[:K]]
        diff = 100
        while diff > 0.0001:
            clusters = []
            num_of_elements = [0 for x in range(K)]
            for sentence in document:
                closest = -1
                point = -1
                for i in range(len(centroids)):
                    dist = np.linalg.norm(sentence-centroids[i])
                    if closest == -1 or dist < closest:
                        closest = dist
                        point = i
                clusters.append( point )
                num_of_elements[point] += 1
            old = centroids
            centroids = [np.zeros(200) for x in range(K)]
            for i in range(len(document)):
                centroids[clusters[i]] += document[i]/num_of_elements[clusters[i]]
            diff = 0
            for i in range(len(centroids)):
                diff += np.linalg.norm(old[i]-centroids[i])
        summary = ""
        for cnt in range(len(centroids)):
            centroid = centroids[cnt]
            closest = -1
            point = -1
            for i in range(len(document)):
                if clusters[i] != cnt:
                    continue
                dist = np.linalg.norm(document[i]-centroid)
                if closest == -1 or closest > dist:
                    closest = dist
                    point = i
            summary+= " " + docs_sentences[mask][point]
        extracted_summaries.append(summary.strip())
    return extracted_summaries

This function simply reads the gold summaries and constructs a list which has the same order of documents as extracted summaries. 

In [8]:
def parse_gold_summary():
    gold_summaries = []
    for article_name in FILE_NAMES:
        summary = open( os.path.abspath( os.path.join( os.getcwd() , GOLD_SUMMARIES_PATH , article_name ) ) , "r" , encoding='latin1' )
        gold_summaries.append(summary.read())
    return gold_summaries

Self explanatory.

In [9]:
def evaluation(gold_summaries, extracted_summaries):
    evaluator = Rouge()
    return evaluator.get_scores(extracted_summaries, gold_summaries, avg=True)

Read the given vector model. Split all files into 10 folds. For each fold, trait it as it is the test set. Since I have no variable which I am training, I don't have validation, simply calculate the scores for each fold seperately. Then take their mean and standart deviation and print them. 

In [10]:
fileObject = open( os.path.abspath( os.path.join( os.getcwd() , VECTOR_PATH ) ) , "rb")
dic = pickle.load(fileObject)
np.random.shuffle(ALL_FILES)
chunks = np.array_split(ALL_FILES,10)
M1rouge = [[] for j in range(3)]
M2rouge = [[] for j in range(3)]
for chunk in chunks:
    FILE_NAMES = chunk
    docs_sentences = [[] for x in range(len(FILE_NAMES))]

    ###MODEL1
    docs_by_sentences = parse_documents_to_sentences()
    docs_as_vectors = sent2vec(docs_by_sentences)
    extracted_summaries = cluster(docs_as_vectors)
    gold_summaries = parse_gold_summary()
    performance = evaluation(gold_summaries, extracted_summaries)
    M1rouge[0].append(list(performance['rouge-1'].values()))
    M1rouge[1].append(list(performance['rouge-2'].values()))
    M1rouge[2].append(list(performance['rouge-l'].values()))

    ###MODEL2
    docs_as_vectors2 = sent2vec2(docs_by_sentences)
    extracted_summaries2 = cluster(docs_as_vectors2)
    performance2 = evaluation(gold_summaries, extracted_summaries2)
    M2rouge[0].append(list(performance2['rouge-1'].values()))
    M2rouge[1].append(list(performance2['rouge-2'].values()))
    M2rouge[2].append(list(performance2['rouge-l'].values()))

print("MEANS:             F          P          R")

print( "Model1 - Rouge1 " + str(np.mean(M1rouge[0], axis=0)) )
print( "Model1 - Rouge2 " + str(np.mean(M1rouge[1], axis=0)) )
print( "Model1 - RougeL " + str(np.mean(M1rouge[2], axis=0)) )

print( "Model2 - Rouge1 " + str(np.mean(M2rouge[0], axis=0)) )
print( "Model2 - Rouge2 " + str(np.mean(M2rouge[1], axis=0)) )
print( "Model2 - RougeL " + str(np.mean(M2rouge[2], axis=0)) )

print("STANDARD DEVIATIONS:")
print( "Model1 - Rouge1 " + str(np.std(M1rouge[0], axis=0)) )
print( "Model1 - Rouge2 " + str(np.std(M1rouge[1], axis=0)) )
print( "Model1 - RougeL " + str(np.std(M1rouge[2], axis=0)) )

print( "Model2 - Rouge1 " + str(np.std(M2rouge[0], axis=0)) )
print( "Model2 - Rouge2 " + str(np.std(M2rouge[1], axis=0)) )
print( "Model2 - RougeL " + str(np.std(M2rouge[2], axis=0)) )

MEANS:             F          P          R
Model1 - Rouge1 [0.52259394 0.63581531 0.45332874]
Model1 - Rouge2 [0.39099371 0.51182168 0.32515498]
Model1 - RougeL [0.47776924 0.62097933 0.44261067]
Model2 - Rouge1 [0.5266946  0.63708816 0.45918922]
Model2 - Rouge2 [0.39543555 0.51262996 0.33129495]
Model2 - RougeL [0.48305029 0.62265397 0.4488084 ]
STANDARD DEVIATIONS:
Model1 - Rouge1 [0.01239938 0.01686186 0.0106851 ]
Model1 - Rouge2 [0.01594904 0.02267762 0.01331539]
Model1 - RougeL [0.0126181  0.01819383 0.01161763]
Model2 - Rouge1 [0.00684145 0.00964196 0.00611342]
Model2 - Rouge2 [0.0079818  0.01134958 0.00701511]
Model2 - RougeL [0.00669358 0.00976949 0.00609237]


#### Conclustion

I was expecting a better improvement moving from model1 to model2. There is a really, really small improvement. I though considering words' tf_idf values would give a better sense to their sentence representation. After all, definitive words will be more dominant than less definitive words. That way semantically close sentences would be closer to each other. I thought the process as eliminating the noise.